In [ ]:
#   Copyright 2025 UKRI-STFC

#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at

#   http://www.apache.org/licenses/LICENSE-2.0

#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.
#
# Authors:
# Franck Vidal (URKI-STFC)

# RXSolutionsReader Laminography Demo

## Data format: RX Solutions

The data is in the format used by devices made by [RX Solutions](https://www.rx-solutions.com/en). The projections are saved in TIFF files. They are flatfield corrected using 16-bit unsigned integers. Metadata is saved in two different files, an XML file that can be used with orbital geometries, and a CSV file that can be used with flexible geometries.

## CIL Version

This notebook was developed using CIL v25.0.0

## Dataset
The data is available from Zenodo: https://doi.org/10.5281/zenodo.??????

It is a laminography dataset of ???. 
It was acquired with the ???? platform developed by [RX Solutions](https://www.rx-solutions.com/en) for the [MATEIS Laboratory](https://mateis.insa-lyon.fr/en) of [INSA-Lyon](https://www.insa-lyon.fr/en/).

Update this filepath to where you have saved the dataset:

In [ ]:
file_path = '/mnt/c/Users/res52584/DATA/DTHE/ZrO2-Cu-1mm-10umvx/geom.csv'

In [ ]:
import numpy as np
import gc

from cil.utilities.display import show2D, show_geometry, show_system_positions
from cil.processors import TransmissionAbsorptionConverter, Slicer, CentreOfRotationCorrector
from cil.framework import ImageGeometry
from cil.plugins.astra import FBP


from cil.utilities.jupyter import islicer, link_islicer

from readers.RXSolutionsDataReader import RXSolutionsDataReader

# Loading Geometry

In [ ]:
reader = RXSolutionsDataReader(file_path, mode="bin", roi={"axis_1": [None, None, 2], "axis_2": [None, None, 2]})

In [ ]:
acq_geom = reader.get_geometry()

In [ ]:
show_system_positions(acq_geom)

In [ ]:
print(acq_geom)

# Loading Projections

In [ ]:
acq_data = reader.read()

In [ ]:
show2D(acq_data, origin='upper-left');

# Pre-processing

In [ ]:
# Prepare the data for Astra
acq_data.reorder(order='astra');

data_exp = TransmissionAbsorptionConverter()(acq_data)

In [ ]:
# Use the system magnification to compute the voxel size
mag = data_exp.geometry.magnification
mean_mag = np.mean(mag)
print("Mean magnification: ", mean_mag)

voxel_size_xy = data_exp.geometry.config.panel.pixel_size[0] / mean_mag
voxel_size_z = data_exp.geometry.config.panel.pixel_size[1] / mean_mag

# Create an image geometry
num_voxel_xy = int(np.ceil(data_exp.geometry.config.panel.num_pixels[0]))
num_voxel_z = int(np.ceil(data_exp.geometry.config.panel.num_pixels[1]))

image_geometry = ImageGeometry(num_voxel_xy, num_voxel_xy, num_voxel_z, voxel_size_xy, voxel_size_xy, voxel_size_z)
print(image_geometry)

# Using a FDK for the reconstruction

In [ ]:
# Reconstruct using FDK
# Instantiate the reconsruction algorithm
fdk = FBP(image_geometry, data_exp.geometry)
fdk.set_input(data_exp)

# Perform the actual CT reconstruction
FDK_recon = fdk.get_output()

## Visualise the reconstruction

In [ ]:
islicer(FDK_recon)

# Using TV regularised least squares solved with FISTA for the reconstruction

In [ ]:
from cil.plugins.astra import ProjectionOperator
from cil.optimisation.functions import LeastSquares
from cil.plugins.ccpi_regularisation.functions import FGP_TV
from cil.optimisation.algorithms import FISTA

projector = ProjectionOperator(image_geometry, data_exp.geometry)
LS = LeastSquares(A=projector, b=data_exp)

alpha = 0.05
TV = FGP_TV(alpha=alpha, nonnegativity=True, device='gpu')
fista_TV = FISTA(initial=FDK_recon, f=LS, g=TV, update_objective_interval=10)

In [ ]:
fista_TV.run(15)
TV_recon = fista_TV.solution

# Compare the two reconstructions

In [ ]:
show2D([FDK_recon, TV_recon], origin='upper-left', slice_list=(('vertical',FDK_recon.shape[0]//2)))

In [ ]:
link_islicer(islicer(FDK_recon), islicer(TV_recon))